# Fetching the values of intermediate layers

Let's recreate environment from previous example.

In [1]:
import torch
import torch.nn as nn

x = [[1,2],[3,4],[5,6],[7,8]]
X = torch.tensor(x).float()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device:", device)
X = X.to(device)

device: cuda


PyTorch provides the fuctionality to fetch the intermediate values of the neural network in two ways:
1. One way is by directly calling layers as if they are functions:

In [2]:
# Setup
class MyNeuralNet1(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)

    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

mynet_1 = MyNeuralNet1().to(device)

# Calling layers directly as if they are functions
input_to_hidden = mynet_1.input_to_hidden_layer(X)
hidden_activation = mynet_1.hidden_layer_activation(input_to_hidden)
print(hidden_activation)

tensor([[0.0000, 0.0000, 0.0000, 0.1653, 0.3287, 0.0000, 0.0000, 1.0586],
        [0.0000, 0.0000, 0.0000, 0.2440, 0.9577, 0.0000, 0.0000, 2.0324],
        [0.0000, 0.0000, 0.0000, 0.3227, 1.5867, 0.0000, 0.0000, 3.0061],
        [0.0000, 0.0000, 0.0000, 0.4014, 2.2157, 0.0000, 0.0000, 3.9799]],
       device='cuda:0', grad_fn=<ReluBackward0>)


Note that we had to call the `input_to_hidden_layer` activation prior to calling `hidden_layer_activation` as the output of `input_to_hidden_layer` is the input to the `hidden_layer_activation` layer.

2. The other way is by specifying the layers that we want to look at in the `forward` method:

In [3]:
# Setup
class MyNeuralNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    
    def forward(self, x):
        hidden1 = self.input_to_hidden_layer(x)
        hidden2 = self.hidden_layer_activation(hidden1)
        output = self.hidden_to_output_layer(hidden2)
        return output, hidden2

# We can access the hidden layer values by indexing into second return value
mynet_2 = MyNeuralNet2().to(device)
print(mynet_2(X)[1]) # [0] is output, [1] is the hidden layer post-activation

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 1.7978, 0.0000, 0.2853, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 3.1215, 0.0000, 1.0852, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 4.4453, 0.0000, 1.8851, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 5.7690, 0.0000, 2.6851, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
